In [1]:
import pandas as pd

movie_day8_df = pd.read_csv('./day8.csv', encoding='euc-kr')

In [2]:
missing_movie_df = pd.read_csv('./8missing.txt', encoding='utf-8')

In [3]:
%%time
new_df = 0
actor_filmo = pd.DataFrame(columns=("actor_name", "filmo","filmo_code"))

for index,row in missing_movie_df.iterrows():
    peopleNm_list = [] ### 매번 비우면서
    
    if '|' in getattr(row,"actor"):
        peopleNm_list = (getattr(row,"actor").split('|'))
    else:
        peopleNm_list.append(getattr(row,"actor"))
        
    filmoNames = getattr(row,"movieNm")
    filmocode = getattr(row,"movie_code")
    
    for actor in peopleNm_list:
        new_df+=1 ## actor_filmo row count
        actor_filmo.loc[new_df] = [actor,filmoNames,filmocode]

Wall time: 1min 52s


In [5]:
actor_code_bk = pd.read_csv('./8actor_code_bak.csv', encoding='utf-8')

In [6]:
%%time
from bs4 import BeautifulSoup
from urllib import parse
import pandas as pd
import html5lib
import re
import time
import aiohttp
import asyncio
import lxml
from aiohttp import ClientSession
from bs4 import BeautifulSoup
import asyncio

async def fetch(link,peopleNm,filmo,index):
    async with ClientSession() as session:
        async with session.get(link) as response:
            html = await response.read()
            try:
                soup = BeautifulSoup(html,"lxml")
                td = soup.find('td', class_='ellipsis')
                link = td.find('a')['onclick']
                code = int(re.search(r'\d+', link).group()) ## 코드(숫자)만 걸러내기
                total_actor_code_df.loc[index] = [peopleNm,filmo,code]
            except Exception as e: 
                pass
    
if __name__ == "__main__":
    count = 1
    length = 300
    tasks = []
    loop = asyncio.get_event_loop()
    
    total_actor_code_df = pd.DataFrame(columns=['actor_name', 'filmo','filmo_code','code'])
    
    for index, row in actor_filmo.iterrows():
        
        peopleNm = getattr(row,"actor_name")
        filmo = getattr(row,"filmo")
        filmocode = getattr(row,"filmo_code")
        
        if any(actor_code_bk['filmo'][actor_code_bk['actor_name']==peopleNm] == filmo):
            total_actor_code_df.loc[index] = [peopleNm,
                                              filmo,
                                              filmocode,
                                              actor_code_bk['code'][actor_code_bk['filmo']==filmo][actor_code_bk['actor_name']==peopleNm].item()
                                             ]
        else:
            peopleNm_parse = parse.quote(peopleNm)
            filmo_parse = parse.quote(filmo)

            param = "sPeopleNm=%s&sMovName=%s" % (peopleNm_parse,filmo_parse)

            url1 = "http://www.kobis.or.kr/kobis/business/mast/peop/searchPeopleList.do?"

            link = url1 + param

            task = asyncio.ensure_future(fetch(link,peopleNm,filmo,index))
            tasks.append(task)

            if count == length: ## Task 300개 채우면
                print(index)
                loop.run_until_complete(asyncio.wait(tasks))
                tasks=[]
                length+= 300
                
            count+=1

        if index == len(actor_filmo): ## 나머지 Task 채우기
            loop.run_until_complete(asyncio.wait(tasks))
            tasks = []
            loop.close()
            

1435
2782
3828
4959
6313
7970
9157
10158
11334
12365
13993
15065
16275
17433
18745
Wall time: 13min 5s


In [8]:
total_actor_code_df.to_csv('./8actor_code.csv', encoding='euc-kr')

In [3]:
import pandas as pd
total_actor_code_df= pd.read_csv('./8actor_code.csv', encoding='euc-kr')

In [4]:
day15_result = pd.read_csv('./15/15_result.csv', encoding='euc-kr')

In [8]:
%%time
import aiohttp
import asyncio
import lxml
from aiohttp import ClientSession
from bs4 import BeautifulSoup
import requests
import pandas as pd

async def fetch(link,actor_code,filmo_name,filmo_code):
    async with ClientSession() as session:
        async with session.get(link) as response:
            html = await response.read()
            
            soup = BeautifulSoup(html,"lxml")

            tbody = soup.findAll('em', class_='tl')

            result = []

            for page in soup.find_all('p',class_='pageList pmt2'):
                page_num = list(page.strings)
                page_result = [i for i in page_num if i != '\n']
                
            for etcParam in page_result:
            
                url2 = "http://www.kobis.or.kr/kobis/business/mast/peop/searchPeopleDtl.do?"

                param = "code=%s&sType=filmo&etcParam=%s"  % (actor_code,etcParam)

                link = url2+param

                average = await get(link,filmo_name,result)

async def get(link,filmo_name,result):
    async with ClientSession() as session:
        async with session.get(link) as response:
            
            html = await response.read()
            
            soup = BeautifulSoup(html,"lxml")

            tr_tag = []
            real_tag = []
            for count in soup.findAll('dl'):
                if any(filmo_name in s for s in list(count.strings)):
                    pass
                elif any('주연' in x for x in list(count.strings)):
    #                 else:
                    tr_tag = tr_tag + list(count.strings)

            for elem in range(0, len(tr_tag)) :
                if tr_tag[elem] == '[공식통계]':
                    real_tag.append(tr_tag[elem+3])

            real_tag = [w.replace('명','') for w in real_tag] ## 명 없애기
            real_tag = [w.replace(',','') for w in real_tag] ## , 없애기
    #             real_tag = [w.replace('','0') for w in real_tag] ## , 없애기
            real_tag = list(filter(None, real_tag))    ## 빈값 지우기
            real_tag = list(map(int,real_tag))

            result.extend(real_tag)
            print(int(average(result)))
            return int(average(result))

def average(values):
    if len(values) == 0:
        return 0
    return sum(values, 0.0) / len(values)
        
if __name__ == "__main__":
    count=1
    length = 300
    tasks = []
    
    
    loop_new = asyncio.get_event_loop()
    test = [10055626,10005276,10029474,10001656,10029595,20134255]
    test2 = []
    for i in test:
        filmo_name = '남한산성'
        actor_code = i
        filmo_code = 20165450
        
        param = "code=%s&sType=filmo"  % (actor_code)

        url1 = "http://www.kobis.or.kr/kobis/business/mast/peop/searchPeopleDtl.do?"

        link = url1 + param

        task = asyncio.ensure_future(fetch(link,actor_code,filmo_name,filmo_code))
        tasks.append(task)
        loop_new.run_until_complete(asyncio.wait(tasks))
        
        if index == length:
            print(index)
            loop_new.run_until_complete(asyncio.wait(tasks))
            tasks = []
            length+=300

        if index == len(total_actor_code_df): 
            loop_new.run_until_complete(asyncio.wait(tasks))
            tasks = []
            loop_new.close()


2674585
2905128
2345931
1898204
2724580
3726561
2912069
1491105
1709868
2009880
1455312
1332055
1345849
913237
838551
0
0
Wall time: 22.9 s


In [18]:
actor_average_df.to_csv('./8actor_average.csv', encoding='euc-kr')

In [5]:
import pandas as pd

movie_day8_df = pd.read_csv('./day8.csv', encoding='euc-kr')

final_df = pd.read_csv('./8actor_average.csv', encoding='euc-kr')

missing_movie_df = pd.read_csv('./8missing.txt')

actor_sum = pd.DataFrame(columns=['movie_name',
                                  'nation',
                                  'distribution',
                                  'director_average',
                                  'actor_sum',
                                  'rate',
                                  'genre',
                                  'release_date',
                                  'day1_screen',
                                  'day1_audience',
                                  'day15_date',
                                  'day15_audience'
                                 ])

for index,row in movie_day8_df.iterrows():
    movie_name = getattr(row, 'movie_name')
    movie_code = getattr(row, 'movie_code')
    director_average = getattr(row,'director_sum')
    nation = getattr(row,'nation')
    release_date = getattr(row,'release_date')
    day1_screen = getattr(row,'day1_screen')
    day1_audience = getattr(row,'day1_audience')
    day15_date = getattr(row,'day8_date')
    day15_audience = getattr(row,'day8_audience')
    
    movie_name = movie_name[1:]
    
    try:
        actor_sum.loc[index] = [
                                movie_name,#                                 'movie_name',
                                nation,#                                   'nation',
                                missing_movie_df['distribution'][missing_movie_df['movie_code'] == movie_code].item(), ## 배급사
                                director_average,#                                   'director_sum',
                                sum(actor_average_df['average_audience'][actor_average_df['filmo'] == movie_name]),#                 'actor_sum'
                                missing_movie_df['rate'][missing_movie_df['movieNm'] == movie_name].item(),#         'rate',
                                missing_movie_df['genre'][missing_movie_df['movieNm'] == movie_name].item(),#        'genre',
                                release_date,#        'release_date'
                                day1_screen,#          'day1_screen',
                                day1_audience, # day1_audience
                                day15_date,
                                day15_audience
                               ]
    except Exception as e:
        print(movie_name,index)
        pass

다크아워 0
밀레니엄 제 1부 : 여자를 증오한 남자들 1
송곳니 2
원더풀 라디오 3
치코와 리타 4
밀레니엄 : 여자를 증오한 남자들 5
초한지 - 천하대전 6
밍크코트 7
세 번째 사랑 8
스탠리의 도시락  9
존 카터: 바숨 전쟁의 서막  10
화차 11
서약 12
가비 13
마이 백 페이지 14
무민가족의 한여름 대소동 15
밀프 16
버니 17
부기우기: 상위 1%의 섹스 18
세상은 가끔 아름답다 19
아웃포스트 : 블랙선 20
어느 예술가의 마지막 일주일 21
월드 워 Z 22
유니버셜 솔저4: 클론의 반란 23
이별계약 24
토토의 움직이는 숲 25
파라다이스 키스 26
퓨어 27
극장판 헌터X헌터 : 팬텀루즈 28
더 웹툰: 예고살인 29
라스트 엑소시즘: 잠들지 않는 영혼 30
빅 웨딩 31
빈센트 32
셀러브리티 섹스테이프 33
엽문4: 종극일전 34
카르페디엠 35
코스모폴리스 36
화이트 하우스 다운 37
힘내세요, 병헌씨 38
가면무도회 39
감시자들 40
론 레인저 41
매니악 : 슬픈 살인의 기록 42
맥베드 43
모니카와의 여름  44
빅 픽처 45
인 더 하우스 46
치어리더 캠프 47
쿵후팬더: 영웅의 탄생 48
타이치0 3D 49
팬텀: 라스트 커맨더 50
48미터 51
라 트라비아타 52
경복 53
넥스트 투모로우 54
마스터  55
명왕성 56
사이드 이펙트 57
슈퍼피쉬 - 끝없는 여정 58
슬립리스 나이트 59
코리올라누스: 세기의 라이벌 60
콩가네 61
퍼시픽 림 62
미스터 고 63
까밀 리와인드 64
레드: 더 레전드 65
브로큰 66
에어라인 플라이트 67
우리는 그곳에 있었다 68
테르마이 로마이 69
토니 스토리: 깡통제국의 비밀 70
영춘권 무림 절대 고수 71
극장판 마법소녀 마도카 마기카(전편): 시작의 이야기 72
극장판 마법소녀 마도카 마기카(후편): 영원의 이야기 73
신검전설 74
그랑블루 리마스터링 감독판 75
극장판 도라에몽: 진구의 비밀도구 박물관 76
더 울버린 77
더 트랩: 난간 

반딧불이의 묘 789
여배우 공작단 790
와즈다 791
익스트랙티드 792
프린스 이고르 793
트랜스포머: 사라진 시대 794
로코왕국의 전설: 엘프킹을 찾아서 795
로크 796
어바웃 체리 797
이것이 우리의 끝이다 798
이브 생 로랑 799
자, 이제 댄스타임 800
커피 한잔이 섹스에 미치는 영향 801
키리시마가 동아리활동 그만둔대 802
프랑스인 김명실 803
소녀괴담 804
나쁜 이웃들  805
내비게이션 806
님포매니악 볼륨2 807
로보싸커 808
마네의 제비꽃 여인: 베르트 모리조 809
블랙딜 810
신의 한 수 811
원나잇 온리 812
천번의 굿나잇 813
카프리치오 814
그 참을 수 없는 맛 815
더 시그널 816
레이드 2 817
나는 야한 여자가 좋다 818
드래프트 데이 819
랄프 스테드먼 스토리: 이상한 나라의 친구들 820
비포 : 치즈와 초콜릿을 찾아서 821
숙희 822
울프 앳 더 도어 823
좋은 친구들 824
테레즈 라캥 825
파이어 : 테러와의 전쟁 826
혹성탈출: 반격의 서막 827
베르테르 828
분신사바 2 829
산타바바라 830
유오성의 7인의 암살단 831
주온 : 끝의 시작 832
중독애 833
파이어스톰 834
극장판 꽃이 피는 첫걸음 : 홈 스위트 홈 835
논픽션 다이어리 836
땡스 포 쉐어링 837
언더 더 스킨 838
위키드 블러드 839
천하무적 키코리키 840
프란시스 하 841
이방인들 842
제7의 봉인 843
컬러풀 844
내 아내의 모든 것 845
돈의 맛 846
레이드 : 첫번째 습격 847
멜랑콜리아 848
미래는 고양이처럼 849
군도: 민란의 시대 850
드래곤 길들이기 2 851
사보타지 852
열애 853
워커즈 854
마담 프루스트의 비밀정원 855
밀애 856
숲의 전설 857
아웃사이더 858
우리가족 859
타이치2 히어로 860
명량 861
벨라 키스 862
가디언즈 오브 갤럭시 863
동경가족 864
머나먼 세상속으로 865
어떤 만남 866


도리화가 1719
열정같은소리하고있네 1720
크림슨 피크 1721
공즉시색 1722
극장판 파워레인저 트레인포스 VS 다이노포스 THE MOVIE 1723
맹글혼 1724
이웃사람 1725
나나나 : 여배우 민낯 프로젝트 1726
레드라이트 1727
제이니 존스 1728
카지노를 털어라 1729
히스테리아 1730
공모자들 1731
서태지밴드 콘서트투어 '콰이어트나이트' 1732
설지 1733
아일랜드 - 시간을 훔치는 섬 1734
어떤이의 꿈 1735
위선자들 1736
트립 투 잉글랜드 1737
한양 기방 춘향뎐 1738
가름워즈: 마지막 예언자 1739
감금의 시간 1740
극적인 하룻밤 1741
나쁜 나라 1742
내 친구의 아내 1743
맥베스 1744
사우스포 1745
스윗 프랑세즈 1746
시카리오: 암살자의 도시 1747
체르노빌: 원전 대폭발 1748
하트 오브 더 씨 1749
아이다 라스칼라 1750
그들이 죽었다 1751
나를 미치게 하는 여자 1752
레전드 1753
리틀 보이 1754
마담 보바리 1755
미스트리스 아메리카 1756
보루토 - 나루토 더 무비 1757
불안한 외출 1758
뽀로로 극장판 컴퓨터 왕국 대모험 1759
서툴지만, 사랑 1760
앨리스: 원더랜드에서 온 소년 1761
캅카 1762
타이밍 1763
타투 1764
파더 앤 도터 1765
파일 : 4022일의 사육 1766
실론, 세렌디피티 1767
대호 1768
히말라야 1769
나의 아들, 나의 어머니 1770
더 봄버 : 최강폭격부대 1771
바닷마을 다이어리 1772
색끈한 퓨전사극 심청야사 1773
스타워즈: 깨어난 포스 1774
락 앤 러브 1775
링컨: 뱀파이어 헌터 1776
미운오리새끼 1777
벨 아미 1778
영건 탐정사무소 1779
인 허 플레이스 1780
어린왕자 1781
포켓몬 더 무비 XY 후파: 광륜의 초마신 1782
944 1783
데싸우 댄서스 1784
마카담 스토리 1785
몬스터 호텔 2 1786
스누피: 더 피너츠 무비 178

나의 위대한 친구, 세잔 2500
내 심장이 건너 뛴 박동 2501
부릉! 부릉! 브루미즈: 스피더의 모험 일기 2502
스노우타임 2503
실버벨 2504
아브릴과 조작된 세계 2505
호두까기 인형: 장난감 왕국 대모험 2506
극장판 도라에몽: 신 진구의 버스 오브 재팬 2507
마스터 2508
씽 2509
업 포 러브 2510
더 모델 2511
에곤 쉴레: 욕망이 그린 그림 2512
위켄즈 2513
포켓몬 더 무비 XY&Z 「볼케니온 : 기계왕국의 비밀」 2514
로그 원: 스타워즈 스토리 2515
루돌프와 많이있어 2516
최후의 Z 2517
하이큐!! 승자와 패자 2518
반드시 크게 들을 것 2 : Wild Days 2519
범죄소년 2520
볼케이노: 삶의 전환점에 선 남자 2521
사이에서 2522
심플 라이프 2523
유신의 추억-다카키 마사오의 전성시대 2524
지저스 크라이스트 슈퍼스타 2525
철가방 우수氏 2526
다윗과 골리앗 2527
사랑은 부엉부엉 2528
파파좀비 2529
내 어깨 위 고양이, 밥 2530
너의 이름은. 2531
눈의 여왕 3: 눈과 불의 마법대결 2532
비밀객잔의 결투 2533
사다코 대 카야코 2534
사랑하기 때문에 2535
여교사 2536
패신저스 2537
걱정말아요 2538
소림사 무림 탐정-취의전장 2539
너브 2540
녹터널 애니멀스 2541
반지의 제왕 : 반지원정대 (확장판) 2542
어쌔신 크리드 2543
얼라이드 2544
모아나 2545
무림강호: 영웅불패의 신화 2546
문영 2547
소시민 2548
요요마와 실크로드 앙상블 2549
처녀 파티 2017 2550
7년-그들이 없는 언론 2551
공조 2552
단지 세상의 끝 2553
더 킹 2554
도쿄 연애사건 2555
터닝메카드W: 블랙미러의 부활 2556
토끼와 거북이 패밀리가 떴다 2557
가디언즈 2558
내 인생의 마지막 변화구  2559
더블 2560
도다리 - 리덕스 2561
다른 길이 있다 2562
로봇 오버로드 2563


In [4]:
actor_sum.to_csv('./8일_최종.csv', encoding='euc-kr')

NameError: name 'actor_sum' is not defined

In [3]:
actor_sum

NameError: name 'actor_sum' is not defined